In [3]:
import os, sys, codecs, time, datetime
import fileinput
import os.path
import stanza
from io import StringIO
from subprocess import call


from docopt import docopt
from propsde.applications.viz_tree import DepTreeVisualizer

import propsde.applications.run as run
import sys
if sys.version_info[0] >= 3:
    unicode = str
    
stanza.download('de')
nlp = stanza.Pipeline('de')

2020-05-12 09:56:42 INFO: Downloading default packages for language: de (German)...
2020-05-12 09:56:43 INFO: File exists: /home/chris/stanza_resources/de/default.zip.
2020-05-12 09:56:48 INFO: Finished downloading models and saved to /home/chris/stanza_resources.
2020-05-12 09:56:48 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| ner       | conll03 |

2020-05-12 09:56:48 INFO: Use device: gpu
2020-05-12 09:56:48 INFO: Loading: tokenize
2020-05-12 09:56:51 INFO: Loading: mwt
2020-05-12 09:56:51 INFO: Loading: pos
2020-05-12 09:56:52 INFO: Loading: lemma
2020-05-12 09:56:52 INFO: Loading: depparse
2020-05-12 09:56:53 INFO: Loading: ner
2020-05-12 09:56:53 INFO: Done loading processors!


In [4]:
import pickle

In [5]:
with open('polizeiberichte.pkl', 'rb') as f:
    data = pickle.load(f, encoding="utf8")

Gute Texte: 5
Schwierige Texte: 11

In [274]:
text_example = data[80]["text1"]
print(text_example)

Gestern Nachmittag kam es zu einem schweren Verkehrsunfall in Rudow. Gegen 17.30 Uhr fuhr ein „Opel“ auf der Schönefelder Straße in Richtung Rudower Spinne. Als die Fahrerin den ersten Ermittlungen zufolge die Kreuzung Neuköllner Straße passieren wollte, übersah sie offenbar einen kreuzenden „Mercedes“, der auf der Neuköllner Straße in Richtung Waltersdorfer Chaussee unterwegs war. Es kam zu einem Zusammenstoß. Der „Mercedes“ geriet ins Schleudern, kippte auf die Fahrerseite und prallte gegen einen geparkten „Seat“. Die Feuerwehr musste den 49 Jahre alten Fahrer aus seinem Fahrzeug bergen und brachte ihn zur stationären Behandlung in ein Krankenhaus. Die 42-jährige „Opel“-Fahrerin blieb unverletzt.


In [174]:
text_example = text_example.replace(". ", ".\n")

In [275]:
print(text_example)

Gestern Nachmittag kam es zu einem schweren Verkehrsunfall in Rudow. Gegen 17.30 Uhr fuhr ein „Opel“ auf der Schönefelder Straße in Richtung Rudower Spinne. Als die Fahrerin den ersten Ermittlungen zufolge die Kreuzung Neuköllner Straße passieren wollte, übersah sie offenbar einen kreuzenden „Mercedes“, der auf der Neuköllner Straße in Richtung Waltersdorfer Chaussee unterwegs war. Es kam zu einem Zusammenstoß. Der „Mercedes“ geriet ins Schleudern, kippte auf die Fahrerseite und prallte gegen einen geparkten „Seat“. Die Feuerwehr musste den 49 Jahre alten Fahrer aus seinem Fahrzeug bergen und brachte ihn zur stationären Behandlung in ein Krankenhaus. Die 42-jährige „Opel“-Fahrerin blieb unverletzt.


In [403]:
file = codecs.open("examples.txt", encoding='utf8')
gs = run.parseSentences(file)

running mate-tools
lemmatizing
parsing + tagging
collapsing
java -jar ext/org.jobimtext.collapsing.jar -i tmp/parsed.conll06 -o tmp -sf -l de -r ext/resources/german_modified.txt -f c -np -nt
UFF


In [404]:
for g,tree in gs: 
    for prop in g.getPropositions('pdf'):
        curProp = '{0}:({1})'.format(prop.pred,
                                      ", ".join([rel + ":" + arg for rel,arg in sorted(prop.args,key=lambda rel:prop.rel_order(rel[0]))]))
        print(curProp)

kommen:(prep_zu:einem schweren Verkehrsunfall in Rudow , mod:Gestern , mod:Nachmittag )
fahren:(subj:ein Opel , prep_gegen:17:30 Uhr , prep_auf:der Schönefelder Straße in Richtung Rudower Spinne )
kreuzenden:(prop_of:einen Mercedes )
sein:(subj:einen Mercedes , prep_auf:der Neuköllner Straße in Richtung Waltersdorfer Chaussee , mod:unterwegs )
offenbar übersehen:(subj:sie , dobj:einen Mercedes , mod:Als die Fahrerin die Kreuzung Neuköllner Straße passieren wollte , mod:den ersten Ermittlungen zufolge )
kommen:(prep_zu:einem Zusammenstoß )
geraten:(subj:Der Mercedes , prep_in:Schleudern )
mussen:(subj:Die Feuerwehr , dobj:den 49 Jahre alten Fahrer , prep_aus:seinem Fahrzeug , mod:bergen )
bringen:(subj:Die Feuerwehr , dobj:ihn , prep_zu:stationären Behandlung , prep_in:ein Krankenhaus )
haben:(subj:er, dobj:Fahrzeug )
und:(conj_und:Die Feuerwehr musste den 49 Jahre alten Fahrer aus seinem Fahrzeug bergen , conj_und:Die Feuerwehr brachte ihn zu stationären Behandlung in ein Krankenhaus )

In [405]:
prop = g.getPropositions('pdf')[0]
print(prop.args)
print(prop.pred)

[['prop_of', 'Die 42-jährige Opel-Fahrerin ']]
unverletzt


In [406]:
test = gs[0][0]
print(test)

Gestern Nachmittag kam es zu einem schweren Verkehrsunfall in Rudow .
873	1,Gestern	ADV	0	0	mod,875
874	2,Nachmittag	ADV	0	0	mod,875
875	3,kam	VVFIN	1	1	
876	4,es	PPER	0	0	dep,875
878	7,schweren	ADJA	0	0	mod,879
879	8,Verkehrsunfall	NN	0	0	prep_zu,875
881	10,Rudow	NE	0	0	prep_in,879



In [501]:
from itertools import compress

def _extract_subj_from_article_stanza(pos_tag, personal_pronouns, pronoun):
    for sent in pos_tag.sentences:
        for word in sent.words:
            if word.deprel == "nsubj" or word.deprel == "root":
                if callable(personal_pronouns[pronoun]):
                    _pronoun = personal_pronouns[pronoun](word.text)
                else:
                    _pronoun = personal_pronouns[pronoun]
                return (_pronoun, word.lemma)
            
def _get_pronoun_for_den(word):
    if word[-2:] == "en":
        return "sie"
    else:
        return "er"

def _map_article_of_subj_to_pronoun(arg):
    personal_pronouns = {"der": "er", "das": "es", "die": "sie", "dem": "er", "den": _get_pronoun_for_den}
    
    if "subj" in arg:
        words = arg[1].lower().split()
        pronoun_place = list(word in personal_pronouns for word in words)
        pronoun = list(compress(words, pronoun_place))
    
        if len(pronoun) == 1:
            pos_tag = nlp(arg[1])
            return _extract_subj_from_article_stanza(pos_tag, personal_pronouns, pronoun[0])
    
    for pronoun in personal_pronouns:
        if pronoun in arg[1] and "prep" not in arg[0]:
            pos_tag = nlp(arg[1])
            return _extract_subj_from_article_stanza(pos_tag, personal_pronouns, pronoun)
                        
    return None

def _get_core_from_noun_phrase(noun_phrase):
    pos_tag = nlp(noun_phrase)
    for sent in pos_tag.sentences:
        for word in sent.words:
            if word.head == 0:
                return {"np": noun_phrase, "core":word.lemma}
            
def _get_context(prop_cache, subjects, words):
    for j in range(0, len(subjects)):
        for k in range(len(subjects[index-j]) - 1, 0, -1):
            if index > 0 and len(subjects[index-j]) > 0 and subjects[index-j][k][0] in words:
                subject_index = words.index(subjects[index-j][k][0])
                return _get_core_from_noun_phrase(subjects[index-j][k][1])["core"]
            
def pretty_print(prop_cache):
    for prop in prop_cache:
        test = {}
        for arg in prop["args"]:
            if "context" in arg[1]:
                test[arg[0]] = arg[1]["context"]
            else:
                test[arg[0]] = arg[1]["core"]
        prep_exists = False
        for k in test:
            if "_" in k:
                if "conj" in k:
                    pass
                else:
                    if "subj" in k:
                        print(test["subj"] + " " + prop["pred"] + " " + k + " " + test[k])
                prep_exists = True
        if not prep_exists:  
            print(test["subj"] + " " + prop["pred"] + " " + test["dobj"])
        
            



In [502]:
subjects = []
prop_cache = []
index = 0

for i, (g, tree) in enumerate(gs):
    for prop in g.getPropositions('pdf'):
        subjects.append([])
        prop_cache.append({"args": prop.args, "pred": prop.pred})
        
while index < len(prop_cache):
    for i, arg in enumerate(prop_cache[index]["args"]):
        words = arg[1].lower().split()

        _article_pronoun = _map_article_of_subj_to_pronoun(arg)
        _noun_phrase_core = _get_core_from_noun_phrase(arg[1])
        if _article_pronoun:
            subjects[index].append(_article_pronoun)
        if _noun_phrase_core:
            prop_cache[index]["args"][i][1] = _noun_phrase_core
        context = _get_context(prop_cache, subjects, words)
        if context:
            prop_cache[index]["args"][i][1]["context"] = context

                    

    index += 1
pretty_print(prop_cache)
print(subjects)

Ermittlung offenbar übersehen Mercedes
Fahrer haben Fahrzeug
[[], [], [], [], [('sie', 'Fahrerin'), ('sie', 'Ermittlung')], [], [('er', 'Mercedes')], [('sie', 'Feuerwehr'), ('er', 'Fahrer')], [('sie', 'Feuerwehr')], [], [('er', 'Feuerwehr')], []]


Der regierende Bürgermeister fährt zum Rathaus.
Im roten Rathaus geht er in den Sitzungssaal.
Der alte Sitzungssaal liegt neben dem Büro von Michael Müller.

In [503]:
for prop in prop_cache:
    str_build = ""
    
    for arg in prop["args"]:
        if arg[0] is not "mod":
            if "context" in arg[1]:
                str_build += arg[1]['context'] + "/"

            str_build += arg[1]['core'] + "/"
    print(prop["pred"] + ": " + str_build)

kommen: Verkehrsunfall/
fahren: Uhr/Opel/Straße/
kreuzenden: Mercedes/
sein: Mercedes/Straße/
offenbar übersehen: Ermittlung/sie/Mercedes/
kommen: Zusammenstoß/
geraten: Mercedes/schleudern/
mussen: Feuerwehr/Fahrer/Fahrzeug/
bringen: Feuerwehr/er/Behandlung/Krankenhaus/
haben: Fahrer/er/Fahrzeug/
und: bergen/bringen/
unverletzt: Opel/


versuchen:(subj:Die Täter , prep_in:der Nacht vom 19 )
Juni 2014:(dobj:die Eingangstür des Bürokomplexes in der Feldtmannstraße gewaltsam zu öffnen )
gelangt:(subj:sie , prep_in:das Objekt , mod:so )
aufhebeln:(subj:sie , dobj:ein Fenster an der Rückseite des Hauses , mod:Als dies nicht gelang , mod:schließlich )
und:(conj_und:Als dies nicht gelang , hebelten sie schließlich ein Fenster an der Rückseite des Hauses auf , conj_und:sie gelangten so in das Objekt )
entwenden:(subj:sie , dobj:auch eine EC-Karte , die bereits an Freitag , 20 , prep_aus:den Büroräumen mehrerer dort ansässiger Firmen , prep_neben:Geld )
20:(subj:auch eine EC-Karte , prep_an:Freitag , mod:bereits )
einsetzen:(obj:Juni 2014 , prep_gegen:5.10 Uhr , prep_an:einem Geldautomaten in der Danziger Straße )

In [504]:
for prop in prop_cache:
    for arg in prop["args"]:
        print(arg[1])

{'np': 'Gestern ', 'core': 'Gestern'}
{'np': 'Nachmittag ', 'core': 'Nachmittag'}
{'np': 'einem schweren Verkehrsunfall in Rudow ', 'core': 'Verkehrsunfall'}
{'np': '17:30 Uhr ', 'core': 'Uhr'}
{'np': 'ein Opel ', 'core': 'Opel'}
{'np': 'der Schönefelder Straße in Richtung Rudower Spinne ', 'core': 'Straße'}
{'np': 'einen Mercedes ', 'core': 'Mercedes'}
{'np': 'einen Mercedes ', 'core': 'Mercedes'}
{'np': 'der Neuköllner Straße in Richtung Waltersdorfer Chaussee ', 'core': 'Straße'}
{'np': 'unterwegs ', 'core': 'unterwegs'}
{'np': 'Als die Fahrerin die Kreuzung Neuköllner Straße passieren wollte ', 'core': 'passieren'}
{'np': 'den ersten Ermittlungen zufolge ', 'core': 'Ermittlung'}
{'np': 'sie ', 'core': 'sie', 'context': 'Ermittlung'}
{'np': 'einen Mercedes ', 'core': 'Mercedes'}
{'np': 'einem Zusammenstoß ', 'core': 'Zusammenstoß'}
{'np': 'Der Mercedes ', 'core': 'Mercedes'}
{'np': 'Schleudern ', 'core': 'schleudern'}
{'np': 'Die Feuerwehr ', 'core': 'Feuerwehr'}
{'np': 'den 49 Jahr

In [494]:
nlp('Die Feuerwehr brachte ihn zu stationären Behandlung in ein Krankenhaus ')

[
  [
    {
      "id": "1",
      "text": "Die",
      "lemma": "der",
      "upos": "DET",
      "xpos": "ART",
      "feats": "Case=Nom|Definite=Def|Gender=Fem|Number=Sing|PronType=Art",
      "head": 2,
      "deprel": "det",
      "misc": "start_char=0|end_char=3"
    },
    {
      "id": "2",
      "text": "Feuerwehr",
      "lemma": "Feuerwehr",
      "upos": "NOUN",
      "xpos": "NN",
      "feats": "Case=Nom|Gender=Fem|Number=Sing",
      "head": 3,
      "deprel": "nsubj",
      "misc": "start_char=4|end_char=13"
    },
    {
      "id": "3",
      "text": "brachte",
      "lemma": "bringen",
      "upos": "VERB",
      "xpos": "VVFIN",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin",
      "head": 0,
      "deprel": "root",
      "misc": "start_char=14|end_char=21"
    },
    {
      "id": "4",
      "text": "ihn",
      "lemma": "er",
      "upos": "PRON",
      "xpos": "PPER",
      "feats": "Case=Acc|Gender=Masc|Number=Sing|Person=3|PronType=Prs",
